This project is for Clean data and analysing the car sales on ebay website.Data set contains data which was available at kaggle website.
The data dictionary provided with data is as follows:
* dateCrawled - When this ad was first crawled. All field-values are taken from this date.
* name - Name of the car.
* seller - Whether the seller is private or a dealer.
* offerType - The type of listing
* price - The price on the ad to sell the car.
* abtest - Whether the listing is included in an A/B test.
* vehicleType - The vehicle Type.
* yearOfRegistration - The year in which the car was first registered.
* gearbox - The transmission type.
* powerPS - The power of the car in PS.
* model - The car model name.
* odometer - How many kilometers the car has driven.
* monthOfRegistration - The month in which the car was first registered.
* fuelType - What type of fuel the car uses.
* brand - The brand of the car.
* notRepairedDamage - If the car has a damage which is not yet repaired.
* dateCreated - The date on which the eBay listing was created.
* nrOfPictures - The number of pictures in the ad.
* postalCode - The postal code for the location of the vehicle.
* lastSeenOnline - When the crawler saw this ad last online.

In [ ]:
import pandas as pd 
import numpy as np 
autos = pd.read_csv(r"C:\Users\ajaya\Downloads\autos.csv")
autos.head(2)


In [ ]:
autos

In [ ]:
autos.info()
autos.head()

Autos dataframe has 20 columns and most of which are string and 371528 entries .some columns have null values .Close to 20% null values.Column names follow  a naming convention called camelcase. We'll start by cleaning the column names to make the data easier to work with.

In [ ]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month','fuel_type', 'brand',
        'unrepaired_damage', 'created_ad', 'num_photos', 'postal_code',
       'last_seen']
autos.head()

In [ ]:
autos.columns


some of the column names has been uppated from camelcase to snowflake.Some column names has been replaced with more meaningful names for better readability and understanding 

# Intial Data Exploration and Cleaning
We will start with data exploration to explore obvious areas where we can do data cleaning

In [ ]:
autos.describe(include = 'all')

Our initial observations:
There are a number of text columns where all (or nearly all) of the values are the same:
* seller
* offer_type
* The num_photos column looks odd. We need to investigate

In [ ]:
autos["num_photos"].value_counts()

We will drop num_photos columns as it has all false values 

In [ ]:

autos["offer_type"].head(10)




Offer type column can be dropped because at has at most one value called Angebot.

In [ ]:
autos["seller"].head(10)


Seller column needs to be dropped as at has at most 1 value 

In [ ]:

autos = autos.drop(["num_photos", "seller", "offer_type"], axis=1)


we dropped below mentioned columns:
* num_photos
* seller
* offer_type

In [ ]:
autos.info()

In [ ]:
autos.describe(include = 'all')

In [ ]:
autos["price"].head(15)

Price and Odometer is numeric column and value is stored as text . We will clean extra character.

In [ ]:
autos['price'] = (autos['price'].astype(str).str.replace("$",""))
autos['price'] = (autos['price'].astype(int))
autos['price'].head()
#autos.describe(include = 'all')


In [ ]:

autos['odometer'].value_counts()

In [ ]:
autos['odometer'] = (autos['odometer'].astype(str).str.replace("km",""))
autos['odometer'] =  (autos['odometer'].astype(int))
autos['odometer'].head()

In [ ]:
autos.rename ({"odometer": "odometer_km"},axis=1,inplace =True)
autos.columns

# Exploring Odometer and Price


In [ ]:
autos["price"].unique().shape
autos["price"].describe()
autos["price"].value_counts().head(20) 
autos["price"].min()
autos["price"].max()



In [ ]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

In [ ]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

In [ ]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

Keep all the Car within range 1 to 350 .And drop remaining values

In [ ]:

autos = autos[autos["price"].between(1,351000)]
autos["price"].describe()


## Exploring date Columns 
Currently there are 5 columns with Date Information:
1.date_crawled
2.registration_year
3.registration_month
4.created_ad
5.last_seen

for few columns we are getting data from crawler and few columns from the website.The non regsitred dates are stored as string.
we will explore each column and learn about listing. 


In [ ]:
autos[['date_crawled','created_ad','last_seen']][0:5]

In [ ]:
(autos["last_seen"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

In [ ]:
(autos["last_seen"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [ ]:
print(autos['created_ad'].str[:10].unique().shape)

In [ ]:
(autos["created_ad"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

In [ ]:
(autos["created_ad"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

There is a large variety of ad created dates. Most fall within 1-2 months of the listing date, but a few are quite old, with the oldest at around 9 months.

In [ ]:
autos["registration_year"].describe()

The year that the car was first registered will likely indicate the age of the car. Looking at this column, we note some odd values. The minimum value is 1000, long before cars were invented and the maximum is 9999, many years into the future.

In [ ]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

Given that this is less than 4% of our data, we will remove these rows.

In [ ]:

autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

It looks like most of the cars were registered in last 20 years

In [ ]:
autos['brand'].describe()

In [ ]:
autos["brand"].value_counts(normalize=True)

German manufacturers represent top five brands, almost 50% of the overall listings. Volkswagen is by far the most popular brand, with approximately double the cars for sale of the next two brands combined.

There are lots of brands that don't have a significant percentage of listings, so we will limit our analysis to brands representing more than 5% of total listings.

In [ ]:
brand_counts = autos["brand"].value_counts(normalize=True)
most_common_brands = brand_counts[brand_counts > .05].index


In [ ]:
brand_mean_prices = {}

for brand in most_common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)

brand_mean_prices
    

Out of top 6 brand there is distnict price gap 
* Volkswagen,BMW,Mercedez benz and Audi is more expensive
* Opel and Ford is less expensive 
* Volkswagen is in between

In [ ]:
bmp_series = pd.Series(brand_mean_prices)
pd.DataFrame(bmp_series ,columns = ["mean_price"])

In [ ]:
brand_mean_mileage = {}

for brand in most_common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

In [ ]:
brand_detail = pd.DataFrame(mean_mileage,columns =['mean_mileage'])
brand_detail

In [ ]:
brand_detail["mean_price"] = mean_prices
brand_detail

The range of car mileages does not vary as much as the prices do by brand, instead all falling within 10% for the top brands. There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.